# 원시 몬테카를로



- 필요성
  - 알파베타법 : 
    - 현 시점에서, 가장 강력한 틱텍토에 적용된 알고리즘
    - 게임의 스케일이 커지면, 막대한 시간이 소요됨
      - 수를 전개하는 부분을 줄여서 상태 가치 계산
        - 원시 몬테카를로 탐색 : Monte Carlo Search
        - 몬테카를로 트리 탐색 : Monte Carlo Tree Search
        - 결론은 램덤 플레이라는 의미
          - 현재 국면에서 게임 종료시까지 계속 랜덤 플레이로 수행.
  - 직접평가함수
    - 프로그래머가 해당 요구사항을 충분히 이해하고
      - 이 지식을 기반으로 평가 함수를 구현

# 기본 게임 환경

In [1]:
# 틱택토 구현
import random

# 게임 상태
class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 취득
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count +=  1
        return count

    # 패배 여부 확인
    def is_lose(self):
        # 돌 3개 연결 여부
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                    self.enemy_pieces[x+y*3] == 0:
                    return False
                x, y = x+dx, y+dy
            return True

        # 배패 여부 확인
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 무승부 여부 확인
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인
    def is_done(self):
        return self.is_lose() or self.is_draw()

    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 선 수 여부 확인
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # 문자열 표시
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

In [2]:
# 랜덤으로 행동 선택
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

In [3]:
# 알파베타법을 활용한 상태 가치 계산
def alpha_beta(state, alpha, beta):
    # 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 무승부 시, 상태 가치 0
    if state.is_draw():
        return  0

    # 합법적인 수의 상태 가치 계산
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha)
        if score > alpha:
            alpha = score

        # 현재 노드의 베스트 스코어가 새로운 노드보다 크면 탐색 종료
        if alpha >= beta:
            return alpha

    # 합법적인 수의 상태 가치의 최대값을 반환
    return alpha

# 알파베타법을 활용한 행동 선택
def alpha_beta_action(state):
    # 합법적인 수의 상태 가치 계산
    best_action = 0
    alpha = -float('inf')
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -float('inf'), -alpha)
        if score > alpha:
            best_action = action
            alpha = score
            
    # 합법적인 수의 상태 가치값 중 최대값을 선택하는 행동 반환
    return best_action

# 플레이아웃

- 현재 국면에서 게임 종료시까지 플레이 하는것
- 게임 종료시까지 둘수 있는  수를 램덤으로  선택하여, 상태가치 
  - 1:승리
  - -1:패배
  - 0:무승부를 반환하는 함수
  - 체스경우 임의의 국면에서 둘수 있는 수는 평균35개이고, 여기서 평균80수 정도에 승부가 난다 
    - 모든 노드를 탐색할려면, 35^80 정도 수를 계산해야함
  - 플레이아웃을 이용하면 회당 80수에서 해결 가능함.

In [5]:
35**80

3353077164822748841188787625923370388228499573110866243697217455549154293219967491214428267909397618495859205722808837890625

In [11]:
# 플레이아웃
def playout(state):
    # 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 무승부 시, 상태 가치 0
    if state.is_draw():
        return  0
    
    # 다음 상태의 상태 평가
    # 비워 있는 돌칸에서 선택가능한것을 찾아서 돌을 넣고 플레이아웃 진행
    return -playout(state.next(random_action(state)))

# 몬테카를로 탐색 행동

In [12]:
# 원시 몬테카를로 탐색을 활용한 행동 선택
def mcs_action(state):
    # 합법적인 수 별로 10회 플레이아웃 시행 후, 상태 가치의 합계 계산
    # 돌을 넣을수 있는 목록만 가져온다
    legal_actions = state.legal_actions()
    # 그 수만큼 0으로 채움
    values        = [0] * len(legal_actions)
    # 한개씩 뺀다
    for i, action in enumerate(legal_actions):
        # 둘수 있는 수별로(돌별로) 10바퀴 돌림 => 임의로 설정 10회
        for _ in range(10):
            # 해당 액션을 놓고 승패 평가 => 합계 평가
            values[i] += -playout(state.next(action))

    # 합법적인 수의 상태 가치 합계의 최대값을 가지는 행동 반환
    return legal_actions[argmax(values)]

# 최대값의 인덱스를 반환
def argmax(collection, key=None):
    # 빈값들 중에서 최대값이 위치한 인덱스를 획득
    return collection.index(max(collection))

# 시뮬레이션

In [13]:
# 원시 몬테카를로 탐색 vs 랜덤/알파베타법의 대전

# 파라미터
# 게임 100회 수행
# 원시 몬테카를로는 랜덤 플레이에 압승
# 원시 몬테카를로는 알파베타법에게는 진다
# -----------------------------------------------------------------------------
EP_GAME_COUNT = 100  # 평가 1회 당 게임 수

# 선 수 플레이어 포인트
def first_player_point(ended_state):
    # 1: 선 수 플레이어 승리, 0: 선 수 플레이어 패배, 0.5: 무승부
    if ended_state.is_lose():
        # 졌는데, 내가 선수면 0점, 적이 졌으면 1
        return 0 if ended_state.is_first_player() else 1
    # 무승부
    return 0.5

In [15]:
# 1 게임 실행
def play(next_actions):
    # 상태 생성
    state = State()

    # 게임 종료 시까지 반복
    while True:
        # 게임 종료 시
        if state.is_done():
            break

        # 행동 얻기
        # 선수면 원시 몬테카를로 탐색
        # 후수면 랜덤 플레이
        next_action = next_actions[0] if state.is_first_player() else next_actions[1]
        action      = next_action(state)

        # 다음 상태 얻기
        state = state.next(action)

    # 선 수 플레이어 포인트 반환
    return first_player_point(state)

In [16]:
# 임의의 알고리즘 평기
def evaluate_algorithm_of(label, next_actions):
    # 여러 차례 대전 반복
    total_point = 0 # 최종 점수
    # 100회 게임 수행
    for i in range(EP_GAME_COUNT):
        # 1 게임 실행, 공정성 순서 변경을 위해서
        if i % 2 == 0:
            # 그냥 수행
            total_point += play(next_actions)
        else:
            # 순서를 뒤집어서 수행 => 1-점수로 표현
            total_point += 1 - play(list(reversed(next_actions)))
        # 출력
        # print의 end에 빈 문자열을 지정하면 같은 위치에서 계속 출력됨
        print('\rEvaluate {}/{}'.format(i + 1, EP_GAME_COUNT), end='')
    print('')

    # 평균 포인트 계산
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

In [17]:
# 원시 몬테카를로 트리 VS. 랜덤
# 1. 알고리즘 준비(함수 2개)
next_actions = (mcs_action, random_action)
# 함수 수동
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

Evaluate 100/100
VS_Random 0.905


In [19]:
# 원시 몬테카를로 트리  VS. 알파베타법
next_actions = (mcs_action, alpha_beta_action)
evaluate_algorithm_of('VS_AlphaBeta {:.3f}', next_actions)

Evaluate 100/100
VS_AlphaBeta 0.260
